### Sedaro Study API Example

This notebook exercises funcionality of the Sedaro Study API tools. The notebook can be pointed at any scenario branch. Examples will step through running a study, downloading the data, and exploring the results.

### ❗ This is an experimental preview of Study functionality.

We are working to add features and streamline usability, so interfaces may change in future versions. Check back for updated versions of this notebook for guidance on the current state of the API.

In [1]:
import json
from sedaro import SedaroApiClient

#### Important: Read Before Running

This notebook requires that you have previously generated an API key in the web UI. That key should be stored in a file called `secrets.json` in the same directory as this notebook with the following format:

```json
{
    "API_KEY": "<API_KEY>"
}
```

API keys grant full access to your repositories and should never be shared. If you think your API key has been compromised, you can revoke it in the user settings interface on the Sedaro website.

In [2]:
# Settings
with open('./secrets.json', 'r') as file:
    API_KEY = json.load(file)['API_KEY']

with open('./config.json', 'r') as file:
    config = json.load(file)

# Obtain a scenario ID from the branch table of a particular scenario.
SCENARIO_BRANCH_ID = config['SUPERDOVE']['SCENARIO_BRANCH_ID']
HOST = config['HOST']

### Running Studies

Studies in Sedaro are collections of individual simulations with varying parameter sets. Currently, variation is limited to the seed for the random number generator. Future versions will offer definition of arbitrary trade studies and covariance analyses.

Starting a study requires at least a target scenario branch ID to simulate and the desired number of iterations.

In [3]:
sedaro = SedaroApiClient(API_KEY, HOST)
study_handle = sedaro.scenario(SCENARIO_BRANCH_ID).study.start(4)

### Downloading and Navigating Results

The study handle created above offers a few methods for interacting data from scenarios. The `.results` method pulls the latest results into a new result object. If the simulation is not complete, the resulting object will indicate the status is "Running" and not contain full results. Alternatively, use the `.results_poll` method to wait for an in-progress simulation to complete and download results after. Both methods allow the user to specify a particular study if the ID is known.

Any object in the results API will provide a descriptive summary of its contents when the `.summarize` method is called.

In [13]:
study = study_handle.results_poll()
study.summarize()

---------------------------------------------------------------------------
                        Sedaro Study Result Summary                        
                        Job ID Nbz8-hhSXL83V_-mn-QsV                       
---------------------------------------------------------------------------
✅ Study succeeded

📋 Study contains 4 simulations

❗ In-memory simulation result caching is ON
---------------------------------------------------------------------------
❓ Query individual simulation results with .result(<ID>)


Since Studies potentially contain a large amount of data, it is important to consider memory usage. A `SedaroStudyResult` will lazily load individual `SedaroSimulationResult` as they are requested and cache them in memory for future requests. The `.set_cache` method controls caching behavior. If caching is disabled, the results will be downloaded each time they are requested memory must be managed manually by the user. With caching enabled, an optional `cache_dir` argument designates an on-disk location for caching simulation result data instead of in-memory. The default is in-memory caching.

In [ ]:
# study.set_cache(cache=True, cache_dir="/path/to/existing/empty/dir")

At any time, the results contained in the local study object can be synced with the server by calling the `.refresh` method. A refresh is only necessary if additional simultions have been triggered for that study or if the study has an in progress status. In all other cases, the data stored locally will remain the same.

In [5]:
study.refresh()

The Study object offers two ways for accessing simulation results: direct indexing and iteration. If results for a particular simulation have not been downloaded yet, a log message will indicate a download is in progress. Both approaches return `SedaroSimulationResult` objects. See the results API demo notebook for guidance on interacting with those results.

If caching is enabled, the operations in the cells below will load from cache after the first run, reducing access time.

In [7]:
sim = study.result(study.job_ids[0])
sim.summarize()

💾 Downloading simulation result id NbuM3NIT8aDuUdRYFDa1-...done!
💾 Successfully saved to cache/sim_NbuM3NIT8aDuUdRYFDa1-.cache
---------------------------------------------------------------------------
                      Sedaro Simulation Result Summary                     
---------------------------------------------------------------------------
✅ Simulation succeeded after 44.8s

🛰️ Templated Agents 
    • SimpleSat
---------------------------------------------------------------------------
❓ Query agent results with .agent(<NAME>)


In [11]:
for sim in study:
    print(sim)

SedaroSimulationResult(branch=Nbp2a45UimH7GlMmpQiBV, status=SUCCEEDED)
SedaroSimulationResult(branch=Nbp2a45UimH7GlMmpQiBV, status=SUCCEEDED)
SedaroSimulationResult(branch=Nbp2a45UimH7GlMmpQiBV, status=SUCCEEDED)
SedaroSimulationResult(branch=Nbp2a45UimH7GlMmpQiBV, status=SUCCEEDED)


The cache, whether in-memory or on-disk, can be cleared with the `.clear_cache` method.

In [12]:
study.clear_cache()